In [11]:
! pip install -U langchain_community tiktoken langchainhub chromadb langchain langgraph tavily-python sentence-transformers

  Using cached tiktoken-0.7.0-cp310-cp310-win_amd64.whl (798 kB)
  Using cached langchainhub-0.1.20-py3-none-any.whl (5.0 kB)
  Using cached langgraph-0.1.1-py3-none-any.whl (87 kB)
  Using cached tavily_python-0.3.3-py3-none-any.whl (5.4 kB)
  Using cached sentence_transformers-3.0.1-py3-none-any.whl (227 kB)
  Using cached regex-2024.5.15-cp310-cp310-win_amd64.whl (268 kB)
  Using cached types_requests-2.32.0.20240622-py3-none-any.whl (15 kB)
  Using cached transformers-4.41.2-py3-none-any.whl (9.1 MB)
     ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
     ---------------------------------------- 0.1/11.0 MB 1.7 MB/s eta 0:00:07
      --------------------------------------- 0.3/11.0 MB 2.6 MB/s eta 0:00:05
     -- ------------------------------------- 0.7/11.0 MB 5.5 MB/s eta 0:00:02
     ----- ---------------------------------- 1.5/11.0 MB 7.8 MB/s eta 0:00:02
     --------- ------------------------------ 2.6/11.0 MB 11.2 MB/s eta 0:00:01
     ------------- ---


[notice] A new release of pip is available: 23.0.1 -> 24.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
! pip install chromadb


[notice] A new release of pip is available: 23.0.1 -> 24.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [13]:
! pip install PyPDF2


[notice] A new release of pip is available: 23.0.1 -> 24.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [14]:
! pip install langchain



[notice] A new release of pip is available: 23.0.1 -> 24.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [15]:
! pip install langchain-community


[notice] A new release of pip is available: 23.0.1 -> 24.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [44]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.prompts import PromptTemplate
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import JsonOutputParser
from langchain.prompts import PromptTemplate
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
import PyPDF2
import os

In [33]:
import PyPDF2

text_list = []
file_path = r"C:\Users\Y Teresha\Desktop\summercourse\apple.pdf"

with open(file_path, 'rb') as pdf_handle:
    pdf_reader = PyPDF2.PdfReader(pdf_handle)
    num_pages = len(pdf_reader.pages)

    text = ''
    for page_num in range(num_pages):
        page = pdf_reader.pages[page_num]
        text += page.extract_text()

    text_list.append(text)

text_list


['UNITED STATES\nSECURITIES AND EXCHANGE COMMISSION\nWashington, D.C. 20549\nFORM 10-K\n(Mark One)\n☒    ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934\nFor the fiscal year ended September\xa024, 2022\nor\n☐    TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934\nFor the transition period from \xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0  to \xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0 .\nCommission File Number: 001-36743\nApple Inc.\n(Exact name of Registrant as specified in its charter)\nCalifornia 94-2404110\n(State or other jurisdiction\nof incorporation or organization)(I.R.S. Employer Identification No.)\nOne Apple Park Way\nCupertino , California 95014\n(Address of principal executive offices) (Zip Code)\n(408) 996-1010\n(Registrant’s telephone number, including area code)\nSecurities registered pursuant to Section 12(b) of the Act:\nTitle of each classTrading \nsymbol(s) Name of each exchange on 

In [35]:
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Create a Document object with the text from the PDF
docs = [Document(page_content=text_list[0])]

# Initialize the RecursiveCharacterTextSplitter with appropriate parameters
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=20,
    length_function=len,
    is_separator_regex=False,
)

# Split the documents into smaller chunks
doc_splits = text_splitter.split_documents(docs)

# Print the split documents
for doc in doc_splits:
    print(doc.page_content)


UNITED STATES
SECURITIES AND EXCHANGE COMMISSION
Washington, D.C. 20549
FORM 10-K
(Mark One)
☒    ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934
For the fiscal year ended September 24, 2022
or
☐    TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934
For the transition period from               to              .
Commission File Number: 001-36743
Apple Inc.
(Exact name of Registrant as specified in its charter)
California 94-2404110
(State or other jurisdiction
of incorporation or organization)(I.R.S. Employer Identification No.)
One Apple Park Way
Cupertino , California 95014
(Address of principal executive offices) (Zip Code)
(408) 996-1010
(Registrant’s telephone number, including area code)
Securities registered pursuant to Section 12(b) of the Act:
Title of each classTrading 
symbol(s) Name of each exchange on which registered
Common Stock, $0.00001  par value per share AAPL The Nasdaq Stock Market LLC
1.000%

In [39]:
# Create a Chroma vector store from the document splits
vectorstore = Chroma.from_documents(
    documents=doc_splits,
    collection_name="rag-chrome",
    embedding=HuggingFaceEmbeddings(),
)

# Initialize the retriever from the vector store
retriever = vectorstore.as_retriever()

c:\Users\Y Teresha\AppData\Local\Programs\Python\Python310\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
c:\Users\Y Teresha\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Y Teresha\.cache\huggingface\hub\models--sentence-transformers--all-mpnet-base-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you 

In [47]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=retriever,
)

question = "What is the revenue of Apple?"
result = qa_chain({"query": question})

print(result["result"])

NameError: name 'llm' is not defined